In [294]:
import numpy as np

In [295]:
class Variable:
    def __init__(self, data):
        if data is not None:
            # data = as_array(data)
            if not isinstance(data, np.ndarray):
                raise TypeError('np.ndarray 자료형을 입력으로 넣어주세요')

        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        funcs = [self.creator]

        # if self.grad == None:
        #     self.grad = np.array(1.0)

        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            if y.grad == None:
                # y.grad = np.array(1.0)
                y.grad = np.ones_like(self.data)
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


In [296]:
class Function():
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        
        return output

    def forward(self, x):
        raise NotImplementedError('순전파')
    
    def backward(self, gy):
        raise NotImplementedError('역전파')

In [297]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [298]:
class Exp(Function):
    def forward(self, input):
        return np.exp(input)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [299]:
def numerical_diff(f, x, eps = 1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [300]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [301]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [302]:
import unittest

class Square_test(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(5.0)
        self.assertEqual(y.data, expected)

In [303]:
x = Variable(np.array(0.5))
print(type(x))
y = square(exp(square(x)))
y.backward()
print(x.grad)

<class '__main__.Variable'>
3.297442541400256


In [304]:
A = square()
B = Exp()
C = square()
x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

print(y.data)
print(y)
print(y.creator)
print(y.grad)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

TypeError: square() missing 1 required positional argument: 'x'

In [ ]:
def backward_test(y):
    func = y.creator
    input = y.creator.input
    print(func)
    print(input.data)
    return func

In [ ]:
y.grad = np.array(1.0)
a = A.backward(y.grad)
b = B.backward(a)
c = C.backward(b)

In [ ]:
def f(x):
    A = square()
    B = Exp()
    C = square()
    return A(B(C(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)

print(dy)

3.2974426293330694


In [ ]:
y.grad = np.array(1.0)
output = C.backward(B.backward(A.backward(y.grad)))
print(output)

3.297442541400256


In [ ]:
f = square()
x = Variable(np.array(2))
dy = numerical_diff(f, x)

print(dy)

4.000000000004


In [ ]:
def f(x):
    A = square()
    B = Exp()
    C = square()
    return A(B(C(x)))

In [ ]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694
